In [1]:
from  __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

/home/jayden/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.utils.vis_utils import plot_model

In [4]:
batch_size = 128 
nb_classes = 10
nb_epoch = 12

# 输入数据的维度
img_rows, img_cols = 28, 28
# 使用的卷积滤波器的数量
nb_filters = 32
# 用于 max pooling 的池化面积
pool_size = (2, 2)
# 卷积核的尺寸
kernel_size = (3, 3)

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


#  如果是theano , 如果是tensorflow
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size, padding = 'valid',
                       input_shape = input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters,kernel_size))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [8]:
plot_model(model, to_file='model-cnn.png')

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 19s - loss: 0.3969 - acc: 0.8781 - val_loss: 0.1071 - val_acc: 0.9658
Epoch 2/12
60000/60000 [==============================] - 6s - loss: 0.1499 - acc: 0.9558 - val_loss: 0.0669 - val_acc: 0.9790
Epoch 3/12
60000/60000 [==============================] - 6s - loss: 0.1115 - acc: 0.9673 - val_loss: 0.0541 - val_acc: 0.9826
Epoch 4/12
60000/60000 [==============================] - 6s - loss: 0.0924 - acc: 0.9724 - val_loss: 0.0443 - val_acc: 0.9849
Epoch 5/12
60000/60000 [==============================] - 7s - loss: 0.0818 - acc: 0.9759 - val_loss: 0.0404 - val_acc: 0.9867
Epoch 6/12
60000/60000 [==============================] - 6s - loss: 0.0750 - acc: 0.9776 - val_loss: 0.0380 - val_acc: 0.9879
Epoch 7/12
60000/60000 [==============================] - 6s - loss: 0.0679 - acc: 0.9798 - val_loss: 0.0359 - val_acc: 0.9881
Epoch 8/12
60000/60000 [==============================] - 6s

In [11]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.03206942018843256
Test accuracy: 0.9893
